In [22]:
from tf.app import use

# Set the path to your local BHSA repository
bhsa_path = "bhsa/tf/2021"

# Define the features you want to load
features = "otype oslots"

# Instantiate Text-Fabric with the local BHSA dataset and specified features
A = use('bhsa', mod=bhsa_path, hoist=globals(), features=features)

# A = use(
#     appName='etcbc/bhsa',
#     # mod='bhsa', x/
#     hoist=globals(), 
#     # checkout='local', 
#     version='2021',
#     # silent=true
#     )


**Locating corpus resources ...**

The requested data is not available offline
	~/text-fabric-data/github/bhsa/tf/2021/2021 not found


Cannot follow https:/github.com/bhsa/tf/releases/latest: HTTP Error 404: Not Found


rate limit is 60 requests per hour, with 59 left for this hour
To increase the rate,see https:/annotation.github.io/text-fabric/tf/advanced/repo.html#github
	connecting to online GitHub repo bhsa/tf ... failed
unexpected error from github.GithubException: 404 {"message": "Not Found", "documentation_url": "https:/docs.github.com/rest/reference/repos#get-a-repository"}


The requested data is not available offline


There were problems with loading data.
The Text-Fabric API has not been loaded!
The app "bhsa" will not work!


In [5]:
A = use('etcbc/bhsa', hoist=globals(), checkout='local', version='2021', silent=False)


**Locating corpus resources ...**

This is Text-Fabric 11.3.1
122 features found and 0 ignored
  3.42s Dataset without structure sections in otext:no structure functions in the T-API
  7.73s All features loaded/computed - for details use TF.isLoaded()
  2.76s All additional features loaded - for details use TF.isLoaded()


Name,# of nodes,# slots/node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


In [10]:
class Word:
    
    id: int = 0                 # e.g., 2
    book: int = 0
    chapter: int = 0
    verse: int = 0
    text: str = ""              # e.g., יִשְׁרְצ֣וּ
    trailer: str = ""
    penalty: float = 0.0        # e.g., 2.4845
    speech: str = ""            # e.g., verb
    person: str = ""            # e.g., p3
    gender: str = ""            # e.g., m
    number: str = ""            # e.g., sg
    verb_tense: str = ""        # e.g., impf
    verb_stem: str = ""         # e.g., piel
    suffix_person: str = ""     # e.g., p3
    suffix_gender: str = ""     # e.g., m
    suffix_number: str = ""     # e.g., sg
    gloss: str = ""             # e.g., swarm
    lex_frequency: int = 0          # e.g., 3403
    occ_frequency: int = 0
    lex_id: int = 0
    
    
BOOK_TO_INDEX = { "Genesis": 1, "Exodus": 2, "Leviticus": 3, "Numbers": 4, "Deuteronomy": 5, "Joshua": 6, "Judges": 7, "Ruth": 8, "1_Samuel": 9, "2_Samuel": 10, "1_Kings": 11, "2_Kings": 12, "1_Chronicles": 13, "2_Chronicles": 14, "Ezra": 15, "Nehemiah": 16, "Esther": 17, "Job": 18, "Psalms": 19, "Proverbs": 20, "Ecclesiastes": 21, "Song_of_songs": 22, "Isaiah": 23, "Jeremiah": 24, "Lamentations": 25, "Ezekiel": 26, "Daniel": 27, "Hosea": 28, "Joel": 29, "Amos": 30, "Obadiah": 31, "Jonah": 32, "Micah": 33, "Nahum": 34, "Habakkuk": 35, "Zephaniah": 36, "Haggai": 37, "Zechariah": 38, "Malachi": 39}

def replace(value):
    if value in {'NA', 'n/a', 'absent'}:
        return None 
    return value 

def add_words_to_database():
    words = []
    for node_id in F.otype.s('word')[:10]:
        word = Word()
        book, chapter, verse = T.sectionFromNode(node_id)
        word.id = node_id
        word.book = BOOK_TO_INDEX[book]
        word.chapter = chapter 
        word.verse = verse
        word.text = F.g_word_utf8.v(node_id)
        word.trailer = F.trailer_utf8.v(node_id)
        word.speech = F.sp.v(node_id)
        word.person = replace(F.prs.v(node_id))
        word.gender = replace(F.gn.v(node_id))
        word.number = replace(F.nu.v(node_id))
        word.verb_tense = replace(F.vt.v(node_id))
        word.verb_stem = replace(F.vs.v(node_id))
        word.suffix_person = replace(F.prs_ps.v(node_id))
        word.suffix_gender = replace(F.prs_gn.v(node_id))
        word.suffix_number = replace(F.prs_nu.v(node_id))
        word.gloss = F.gloss.v(node_id)
        word.lex_frequency = F.freq_lex.v(node_id)
        word.occ_frequency = F.freq_occ.v(node_id)
        word.lex_id = L.u(node_id, otype='lex')[0]
        words.append(word)
    return words

words = add_words_to_database()


In [11]:
for word in words:
    print(vars(word))

{'id': 1, 'book': 1, 'chapter': 1, 'verse': 1, 'text': 'בְּ', 'trailer': '', 'speech': 'prep', 'person': None, 'gender': None, 'number': None, 'verb_tense': None, 'verb_stem': None, 'suffix_person': None, 'suffix_gender': None, 'suffix_number': None, 'gloss': 'in', 'lex_frequency': 15542, 'occ_frequency': 14194, 'lex_id': 1437602}
{'id': 2, 'book': 1, 'chapter': 1, 'verse': 1, 'text': 'רֵאשִׁ֖ית', 'trailer': ' ', 'speech': 'subs', 'person': None, 'gender': 'f', 'number': 'sg', 'verb_tense': None, 'verb_stem': None, 'suffix_person': None, 'suffix_gender': None, 'suffix_number': None, 'gloss': 'beginning', 'lex_frequency': 51, 'occ_frequency': 45, 'lex_id': 1437603}
{'id': 3, 'book': 1, 'chapter': 1, 'verse': 1, 'text': 'בָּרָ֣א', 'trailer': ' ', 'speech': 'verb', 'person': None, 'gender': 'm', 'number': 'sg', 'verb_tense': 'perf', 'verb_stem': 'qal', 'suffix_person': None, 'suffix_gender': None, 'suffix_number': None, 'gloss': 'create', 'lex_frequency': 48, 'occ_frequency': 15, 'lex_id'

In [7]:
print(F.f.v(1))

בְּ


In [8]:
from tf.fabric import Fabric

path = "bhsa/tf/2021"
features = 'sp prs gn nu vt vs prs_ps prs_gn prs_nu gloss freq_lex freq_occ'

TF = Fabric(locations=path)
api = TF.load(features=features)
api.makeAvailableIn(globals())
# A = use('data:bhsa', hoist=globals())

   |     1.69s T freq_lex             from bhsa/tf/2021
   |     1.75s T freq_occ             from bhsa/tf/2021
   |     2.02s T gloss                from bhsa/tf/2021
   |     1.71s T gn                   from bhsa/tf/2021
   |     1.72s T prs                  from bhsa/tf/2021
   |     1.69s T prs_gn               from bhsa/tf/2021
   |     1.70s T prs_nu               from bhsa/tf/2021
   |     1.71s T prs_ps               from bhsa/tf/2021
   |     1.76s T sp                   from bhsa/tf/2021
   |     1.76s T vs                   from bhsa/tf/2021
   |     1.72s T vt                   from bhsa/tf/2021


[('Computed',
  'computed-data',
  ('C Computed', 'Call AllComputeds', 'Cs ComputedString')),
 ('Features', 'edge-features', ('E Edge', 'Eall AllEdges', 'Es EdgeString')),
 ('Fabric', 'loading', ('TF',)),
 ('Locality', 'locality', ('L Locality',)),
 ('Nodes', 'navigating-nodes', ('N Nodes',)),
 ('Features',
  'node-features',
  ('F Feature', 'Fall AllFeatures', 'Fs FeatureString')),
 ('Search', 'search', ('S Search',)),
 ('Text', 'text', ('T Text',))]

In [19]:
for word in words:
    for key, value in vars(word).items():
        print(key, ' : ', value)

id  :  1
book  :  1
chapter  :  1
verse  :  1
text  :  בְּ
trailer  :  
speech  :  prep
person  :  None
gender  :  None
number  :  None
verb_tense  :  None
verb_stem  :  None
suffix_person  :  None
suffix_gender  :  None
suffix_number  :  None
gloss  :  in
frequency  :  15542
lexeme_id  :  1437602
id  :  2
book  :  1
chapter  :  1
verse  :  1
text  :  רֵאשִׁ֖ית
trailer  :   
speech  :  subs
person  :  None
gender  :  f
number  :  sg
verb_tense  :  None
verb_stem  :  None
suffix_person  :  None
suffix_gender  :  None
suffix_number  :  None
gloss  :  beginning
frequency  :  51
lexeme_id  :  1437603
id  :  3
book  :  1
chapter  :  1
verse  :  1
text  :  בָּרָ֣א
trailer  :   
speech  :  verb
person  :  None
gender  :  m
number  :  sg
verb_tense  :  perf
verb_stem  :  qal
suffix_person  :  None
suffix_gender  :  None
suffix_number  :  None
gloss  :  create
frequency  :  48
lexeme_id  :  1437604
id  :  4
book  :  1
chapter  :  1
verse  :  1
text  :  אֱלֹהִ֑ים
trailer  :   
speech  :  subs
pe